# 分类猫和狗的图片
使用一个还不错的相机采集图片 (12M像素)
RGB 图片有36M元素
使用100大小的单隐藏层MLP，模型有3.6B元素
远多于世界上所有猫和狗总数 (900M 狗，600M 猫)

# 重新考察全连接层
将输入和输出变形为矩阵 (宽度，高度)
将权重变形为4-D张量 (h,w) 到(h',w')
$$
h_{i,j}=\sum_{k,l}w_{i,j,k,l}x_{k,l}=\sum_{a,b}\nu_{i,j,a,b}x_{i+a,j+b}
$$
V是W 的重新索引$v_{i,j,a,b}=w_{i,j,i+a,j+b}$

# 原则 #1- 平移不变性
x的平移导致h 的平移 $h_{i,j}=\displaystyle\sum_{a,b}\nu_{i,j,a,b}x_{i+a,j+b}$
不应该依赖于(i,j)
解决方案: $v_{i,j,a,b}=v_{a,b}$
$h_{i,j}=\sum_{a,b}\nu_{a,b}x_{i+a,j+b}$
这就是2维(卷积) 交叉相关

# 原则 \#2-局部性
$$h_{i,j}=\sum_{a,b}\nu_{a,b}x_{i+a,j+b}\\$$
$\text{当评估 }h_{i,j}\text{时,我们不应该用远离 }x_{i,j}\text{的参数}\\$
$\text{解决方案:当}|a|,|b|>\Delta\text{ 时,使得 }\nu_{a,b}=0$
$$h_{i,j}=\sum_{a=-\Delta}^\Delta\sum_{b=-\Delta}^\Delta\nu_{a,b}x_{i+a,j+b}$$

# 总结·
对全连接层使用平移不变性和局部性得到卷积层
$$h_{i,j}=\sum_{a,b}\nu_{i,j,a,b}x_{i+a,j+b}$$
$$\Downarrow$$
$$h_{i,j}=\sum_{a=-\Delta}^{\Delta}\sum_{b=-\Delta}^{\Delta}\nu_{a,b}x_{i+a,j+b}$$

# 卷积层

# 二维卷积层
·输入$\mathbf{x}:n_{h}\times n_{w}$
·核$\mathbf{w}:k_{h}\times k_{w}$
 ·偏差$b\in\mathbb{R}$
 ·输出$\mathbf{Y}:(n_{h}-k_{h}+1)\times(n_{w}-k_{w}+1)$
 $$\mathbf{Y}=\mathbf{X}\star\mathbf{W}+b$$
 ·$\mathbf{W}$和$b$是可学习的参数

# 交叉相关vs卷积
二维交叉相关
$$y_{i,j}=\sum_{a=1}^{h}\sum_{b=1}^{w}w_{a,b}x_{i+a,j+b}$$
二维卷积
$$y_{i,j}=\sum_{a=1}^{h}\sum_{b=1}^{w}w_{-a,-b}x_{i+a,j+b}$$
由于对称性，在实际使用中没有区别

# 一维和三维交叉相关
·一维
$$y_i=\sum_{a=1}^hw_ax_{i+a}$$
·文本
·语言
·时序序列

·三维
$$y_{i,j,k}=\sum_{a=1}^{h}\sum_{b=1}^{w}\sum_{c=1}^{d}w_{a,b,c}x_{i+a,j+b,k+c}$$
·视频
·医学影像
·气象地图

# 总结
卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
核矩阵和偏移是可学习的参数
核矩阵的大小是超参数

# 图像卷积

In [1]:
import torch
from torch import nn
from d2l import torch as d2l


def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

验证上述二维互相关运算的输出

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积层的一个简单应用：检测图像中不同颜色的边缘

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
K = torch.tensor([[1.0, -1.0]])

输出Y中的1代表从白色到黑色边缘，-1代表从黑色到白色的边缘

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只可以检测垂直边缘

In [7]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由X生成Y的卷积核

In [11]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i + 1}, loss {l.sum():.3f}')

batch 2, loss 4.301
batch 4, loss 0.775
batch 6, loss 0.152
batch 8, loss 0.034
batch 10, loss 0.009
batch 12, loss 0.003
batch 14, loss 0.001
batch 16, loss 0.000
batch 18, loss 0.000
batch 20, loss 0.000


所学的卷积核权重张量

In [12]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0008, -0.9990]])